In [2]:
import numpy as np 
import math
import cmath
import scipy.fftpack

# To Ask, signal in frequency domain or time domain ? if Time how to transorm to frequency ? Is it fourier transform ? 
# I used Fourier Transform to use the signal

# Should I rescale the points to have them in a unit circle ?
# Does the rfftfreq give the frequencies used in the fourier transform rfft 
# The length of the array given by rfftfreq is (n/2) + 1, last week you used 128 samples as an example so should I use 254 to have 128 frequency samples or just use 128 in the rfftfreq function directly and have 65 frequency samples 
# What tests could I do to see if my forward model is right
# How to read the lego thing 

def forward(sourceSignal, numberOfMicrophones, microphonesPosition, minDegree = 0, maxDegree = 360, step = 1):
    
    ''' Forward Model
 
     Input:
       - sourceSignal: Signal in a discrete form in the time domain as a numpy matrix (vector in reality)
       - minDegree : the degree minimal in which we want to see the signal received (included)
       - maxDegree : the degree maximal in which we want to see the signal received (excluded)
       - step : the step we want between each direction
       - numberOfMicrophones: number of microphones present in the model
       - microphonesPosition: the positions of the microphones 
    
     Output:
       - y: signals received by the microphones in each degrees from minDegree to maxDegree by doing a step of step degrees
    '''
        
    for i in range(minDegree, maxDegree, step):
        if i == minDegree:
            y = [forward_degree(sourceSignal, i, numberOfMicrophones, microphonesPosition)]
        else:
            y.append(forward_degree(sourceSignal, i, numberOfMicrophones, microphonesPosition))
        

    
    
    return np.array(y)

def forward_degree(sourceSignal, thetaSignal, numberOfMicrophones, microphonesPosition):
    
    ''' Forward Model
 
     Input:
       - sourceSignal: Signal in a discrete form in the time domain as a numpy matrix (vector in reality)
       - thetaSignal: The direction  degree of the source (in degrees not radians)
       - numberOfMicrophones: number of microphones present in the model
       - microphonesPosition: the positions of the microphones 
    
     Output:
       - y: signals received by the microphones
    '''
    
    micPosX = np.zeros(numberOfMicrophones)
    micPosY = np.zeros(numberOfMicrophones)
    
    for i in range(numberOfMicrophones):
        (micPosX[i], micPosY[i]) = microphonesPosition[i]
    
    # We calculate the distances between the source and the different microphones
    # This is an hypothetic distance, as the micrphone is considered "far away", using the direction of the signal
    
    distanceSourceMicrophones = np.zeros(numberOfMicrophones)
    
    for i in range(numberOfMicrophones):
        horizontalDistance = micPosX[i] - math.cos(math.radians(thetaSignal))
        verticalDistance = micPosY[i] - math.sin(math.radians(thetaSignal))
        distanceSourceMicrophones[i] = math.sqrt(math.pow(horizontalDistance, 2) + math.pow(verticalDistance, 2))
            
    
    sample_rate = 8000 # The maximum frequency we want to use
    sample_number = 256 # The number of points we will have
    freq = np.fft.rfftfreq(sample_number, d=1./(2*sample_rate))
    
    sourceFourier = np.fft.rfft(sourceSignal, sample_number)
    (sizeX, )  = sourceFourier.shape
    
    hFunction = np.zeros((numberOfMicrophones, sizeX), dtype=np.complex_)
    c = 340.29 #speed of sound 340.29 m/s

        
    y = np.zeros((numberOfMicrophones, sizeX), dtype=np.complex_)

    #The different filter signals use the distance : H(w) = exp(-j*w*tau)
    for i in range(numberOfMicrophones):
        tau = distanceSourceMicrophones[i] / c
        for k in range(sizeX):
            hFunction[i][k] = cmath.exp(-freq[k]*1j*tau)
            y[i][k] = sourceFourier[k]*hFunction[i][k]


    return y


y = forward(np.array([1,5,6,8,2]), 3, [(1, 0.5), (0.8,0.33), (0.6,0.8)])
y.shape



(360, 3, 129)